# 🛫 External Stuff

## 🚗 DMVs

In [121]:
set nocount on; 
select * from sys.database_scoped_credentials;
select * from sys.external_data_sources;
select * from sys.external_file_formats;
select * from sys.external_tables;

## ⚓ Infrastructure 

In [139]:
if exists (select * from sys.external_tables where name ='extBand1') drop external table extBand1;
if exists (select * from sys.external_tables where name ='extBand2') drop external table extBand2;
if exists (select * from sys.external_tables where name ='extBand3') drop external table extBand3;
if exists (select * from sys.external_tables where name ='cetasBand') drop external table cetasBand;
if exists (select * from sys.external_tables where name ='cetasDevs') drop external table cetasDevs;

if exists (select 1 from sys.external_file_formats where name ='parquet_ff')            drop external file format parquet_ff;
if exists (select 1 from sys.external_file_formats where name ='csv2_ff')               drop external file format csv2_ff;
if exists (select 1 from sys.external_file_formats where name ='csv_ff')                drop external file format csv_ff;
if exists (select 1 from sys.external_data_sources where name ='hiramblob_ds')          drop external data source hiramblob_ds;
if exists (select 1 from sys.database_scoped_credentials where name ='hiramblob_cred')  drop database scoped credential hiramblob_cred;

Commands completed successfully.

Total execution time: 00:00:02.135

## 🐱‍👤 Don't forget my 🔑

In [123]:
create database scoped credential hiramblob_cred
with 
     identity = 'sqlva4onabrnvvfrkw',
     secret = 'FbSXN7aosiHHgy9qmvVMZsQXMg1YIdoocMTIaVrUJKt+dr+U1SjLju9advAdsnbv/m8dcmqLqJwyDkyFR/uNAA==' --mykey
;

create external data source hiramblob_ds
with (    
    location = 'wasbs://hiramblob@sqlva4onabrnvvfrkw.blob.core.windows.net/',
    credential = hiramblob_cred,
    type = HADOOP
);

## abfs uri for gen2

In [1]:
create database scoped credential lakehiram_cred
with 
     identity = 'lakehiram',
     secret = '' --mykey
;

create external data source lakehiram_prod
with (    
    location = 'abfss://prod@lakehiram.dfs.core.windows.net/',
    credential = lakehiram_cred,
    type = HADOOP
);

Commands completed successfully.

Total execution time: 00:00:00.249

## 📝 Files Formats

In [4]:
if exists (select 1 from sys.external_file_formats where name = 'parquet_ff') drop external file format parquet_ff;
create external file format parquet_ff
with (
    format_type = parquet,  
    data_compression = 'org.apache.hadoop.io.compress.SnappyCodec'  
);

if exists (select 1 from sys.external_file_formats where name = 'csv2_ff') drop external file format csv2_ff;
create external file format csv2_ff
with (
    format_type = delimitedtext,
    format_options(
        field_terminator = ',',
        string_delimiter = '"',
        first_row = 2, 
        use_type_default = true
    )
);

if exists (select 1 from sys.external_file_formats where name = 'csv_ff') drop external file format csv_ff;
create external file format csv_ff
with (
    format_type = delimitedtext,
    format_options(
        field_terminator = ',',
        string_delimiter = '"',
        use_type_default = true
    )
);

Commands completed successfully.

Total execution time: 00:00:01.040

## 🎁 CETAS - Create External Table As Select (out-bound)

In [138]:
if exists (select * from sys.external_tables where name ='extBand1') drop external table extBand1;
if exists (select * from sys.external_tables where name ='extBand2') drop external table extBand2;
if exists (select * from sys.external_tables where name ='extBand3') drop external table extBand3;
if exists (select * from sys.external_tables where name ='cetasBand') drop external table cetasBand;
if exists (select * from sys.external_tables where name ='cetasDevs') drop external table cetasDevs;

Commands completed successfully.

Total execution time: 00:00:00.718

In [126]:
create external table extBand1 
-- (
--     BandId  int,
--     Band    nvarchar(50),
--     Album   nvarchar(50)
-- )
with (   
    location = 'extBand1',  
    data_source = hiramblob_ds,  
    file_format = parquet_ff  
)
as 
    select * from dbo.Band;
go

create external table extBand2
with (   
    location = 'extBand2',  
    data_source = hiramblob_ds,  
    file_format = csv_ff  
)
as 
    select * from dbo.Band;
go

create external table extBand3
with (   
    location = 'extBand3',  
    data_source = hiramblob_ds,  
    file_format = csv2_ff  
)
as 
    select * from dbo.Band;

In [5]:
create external table extBand4
with (   
    location = 'extBand4',  
    data_source = lakehiram_prod,  
    file_format = csv_ff  
)
as 
    select * from dbo.Band;
go

(119955 rows affected)

Total execution time: 00:00:21.232

In [127]:
exec sp_spaceused Band;
select count(*) from extBand1 union
select count(*) from extBand2 union
select count(*) from extBand3

select top 3 * from extBand1 where BandID between 1 and 3 union
select top 3 * from extBand2 where BandID between 1 and 3 union
select top 3 * from extBand3 where BandID between 1 and 3 order by bandid ;

## 🔎 CETAS - Create External Table As Select (in-bound) 
##### https://github.com/microsoft/sql-server-samples/tree/master/samples/features/sql-big-data-cluster/data-virtualization/storage-pool

In [128]:
if exists (select * from sys.external_tables where name ='cetasBand') drop external table cetasBand;
if exists (select * from sys.external_tables where name ='cetasDevs') drop external table cetasDevs;

In [129]:
create external table cetasBand (
    BandId  int,
    Band    nvarchar(50),
    Album   nvarchar(50)
)
with (
    location = '/extBand3/',
    data_source = hiramblob_ds,
    file_format = csv2_ff,
    reject_type = value,
    reject_value = 0
);

create external table cetasDevs (
    first   varchar(128),
    last    varchar(128)
)
with (
    location = 'devs.csv',
    data_source = hiramblob_ds,
    file_format = csv2_ff,
    reject_type = value,
    reject_value = 0
);


In [130]:
sp_spaceused cetasBand
go 
sp_spaceused cetasDevs
go

select count(*) from cetasBand;
select count(*) from cetasDevs;
select top 100 * from cetasBand order by BandId;
select top 100 * from cetasDevs;

## 💀 DML Operations

In [136]:
insert cetasDevs select 'ocean','fleitas';


In [137]:
select * from cetasDevs
go
delete from cetasDevs where first='hiram'
go
insert cetasDevs select 'ocean','fleitas';

--not needed on ASDW
--sp_configure 'allow polybase export', 1; reconfigure; 